# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [12]:
#Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [11]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [16]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    tables = pd.read_html(html)
    for df in tables:
        if len(df.columns) >= 3:
            return flatten_headers(df)
    raise ValueError("No table with at least 3 columns found.")


def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    # Strip spaces from all columns
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

    # Convert Alpha-2 and Alpha-3 to uppercase
    for col in ["Alpha-2", "Alpha-3"]:
        if col in df.columns:
            df[col] = df[col].str.upper()

    # Cast Numeric to int (nullable)
    if "Numeric" in df.columns:
        df["Numeric"] = pd.to_numeric(df["Numeric"], errors="coerce").astype("Int64")

    # Drop rows with invalid values (NaN in Numeric after coercion)
    df.dropna(subset=["Numeric"], inplace=True)

    return df


def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    return df.sort_values(by="Numeric", ascending=False).head(top)

In [17]:
# Q1 — Write your answer here
url = "https://www.iban.com/country-codes"
html = fetch_html(url)

df_raw = q1_read_table(html)
df_clean = q1_clean(df_raw)
df_top = q1_sort_top(df_clean, 15)

print(df_top)
df_clean.to_csv("data_q1.csv", index=False)
print("\n Saved as data_q1.csv")


                                               Country Alpha-2 code  \
247                                             Zambia           ZM   
246                                              Yemen           YE   
192                                              Samoa           WS   
244                                  Wallis and Futuna           WF   
240                 Venezuela (Bolivarian Republic of)           VE   
238                                         Uzbekistan           UZ   
237                                            Uruguay           UY   
35                                        Burkina Faso           BF   
243                              Virgin Islands (U.S.)           VI   
236                     United States of America (the)           US   
219                       Tanzania, United Republic of           TZ   
108                                        Isle of Man           IM   
113                                             Jersey           JE   
92    

/tmp/ipython-input-3673404506.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [19]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    soup = BeautifulSoup(html, 'lxml')
    items = []
    for athing in soup.select('.athing'):
        rank = athing.select_one('.rank').text.strip('.')
        title = athing.select_one('.titleline a').text
        link = athing.select_one('.titleline a')['href']

        subtext = athing.find_next_sibling('tr').select_one('.subtext')
        points = subtext.select_one('.score')
        points = points.text.split()[0] if points else '0'

        comments = subtext.select_one('.subtext a:last-child')
        comments = comments.text.split()[0] if comments and 'comment' in comments.text else '0'

        user = subtext.select_one('.hnuser')
        user = user.text if user else ''

        items.append({
            'rank': rank,
            'title': title,
            'link': link,
            'points': points,
            'comments': comments,
            'user': user
        })

    return pd.DataFrame(items)


def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    for col in ['rank', 'points', 'comments']:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    for col in ['title', 'link', 'user']:
        df[col] = df[col].fillna('').astype(str)

    return df


def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    return df.sort_values(by='points', ascending=False).head(top)

In [21]:
# Q2 — Write your answer here
url = "https://news.ycombinator.com/"
html = fetch_html(url)

df_raw = q2_parse_items(html)
df_clean = q2_clean(df_raw)
df_top = q2_sort_top(df_clean, 15)

print(df_top)
df_clean.to_csv("data_q2.csv", index=False)
print("\n Saved as data_q2.csv")



    rank                                              title  \
10    11  YouTube Removes Windows 11 Bypass Tutorials, C...   
8      9                            Why I love OCaml (2023)   
25    26  VLC's Jean-Baptiste Kempf Receives the Europea...   
26    27                              James Watson has died   
4      5  Myna: Monospace typeface designed for symbol-h...   
0      1                                Why is Zig so cool?   
7      8                             Ruby Solved My Problem   
6      7                                How did I get here?   
3      4                       Becoming a Compiler Engineer   
1      2  Snapchat open-sources Valdi a cross-platform U...   
20    21                     Angel Investors, a Field Guide   
23    24  FAA to restrict commercial rocket launches to ...   
16    17                            Venn Diagram for 7 Sets   
17    18  Transducer: Composition, abstraction, performa...   
2      3       Mullvad: Shutting down our search proxy 